In [3]:
!python -V

Python 3.11.7


In [4]:
import pandas as pd

In [5]:
import pickle

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [8]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    std = df.duration.std()
    before_len = len(df)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    after_len = len(df)
    percentile = after_len/before_len
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return (percentile,std,df)

In [9]:
(percentile,std,df) = read_dataframe('./data/yellow_tripdata_2023-01.parquet')

In [10]:
percentile

0.9812202822125979

In [11]:
std

42.594351241920904

In [12]:
df.shape[1] -1

19

In [13]:
df['duration'].std()

9.939385620145579

In [14]:
categorical = ['PULocationID', 'DOLocationID']

train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

7.649262029221019

In [16]:
X_train.shape[1]

515

In [17]:
mean_squared_error(y_train, y_pred, squared=False)

7.649262029221019

In [40]:
(_,_,df_val) = read_dataframe('./data/yellow_tripdata_2023-02.parquet')
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [41]:
y_val_pred =lr.predict(X_val) 

In [42]:
y_val_actual = df_val[target].values

In [43]:
mean_squared_error(y_val_actual, y_val_pred, squared=False)

7.8118169669491095